# Imports 

In [102]:
import yt_dlp
from bs4 import BeautifulSoup
import urllib.request
import os
import numpy as np
import cv2
from time import time as timer
import sys
import pandas as pd
from sklearn.model_selection import train_test_split

## Get URLS

In [14]:
def SearchVid(search):
    responce = urllib.request.urlopen('https://www.youtube.com/results?search_query='+search)

    soup = BeautifulSoup(responce)    
    divs = soup.find_all("div", { "class" : "yt-lockup-content"})


    for i in divs:
        href= i.find('a', href=True)
        print(href.text,  "\nhttps://www.youtube.com"+href['href'], '\n')
        with open(SearchString.replace("%20", "_")+'.txt', 'a') as writer:
            writer.write("https://www.youtube.com"+href['href']+'\n')

print("What are you looking for?")
SearchString = input()
SearchVid(SearchString.replace(" ", "%20"))

What are you looking for?
wrong-way drivers


In [ ]:
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/results?search_query=dashcam+trip&sp=EgIYAQ%253D%253D'])

## Rename Files 

In [12]:
from os import listdir
from os.path import isfile, join

In [15]:
def rename(name,path):
    mypath= path
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    i=0
    for f in onlyfiles:
        old_file = os.path.join(mypath, str(f))
        new_file = os.path.join(mypath,name+f'_{i}.mp4' )
        os.rename(old_file, new_file)
        i+=1
        

In [45]:
rename('no_wr_way','/Users/workplace/Documents/GitHub/data/trimmed')

## Extract Frames 

In [46]:
# setup paths
FPS_OUTPUT = 1
pwd = '/Users/workplace/Documents/GitHub/data/'
path_videos = pwd + 'trimmed/'
path_data = pwd + 'no_wr_way/'

In [ ]:
# read video paths
paths = os.listdir(path_videos)
paths = [path_videos + v for v in paths if v != '.DS_Store']
paths

In [ ]:
# create data folder
if not os.path.exists(path_data):
    os.makedirs(path_data)

for c, path in enumerate(paths):

    # extract video filename from path
    filename = path.split("/")[-1].split(".")[0]
    print(filename)

    if not os.path.exists(path_data + filename):

        print("Extracting frames from video {}/{}: {}".format(c+1,len(paths),filename))

        # create directory for this video's frames
        os.makedirs(path_data + filename)

        # open video
        vidcap = cv2.VideoCapture(path)

        # get fps
        print("video FPS {}".format(vidcap.get(cv2.CAP_PROP_FPS)))
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        fps_savecheck = fps/FPS_OUTPUT

        # read frames and save to images at fps_save
        success,image = vidcap.read()
        count_frame = 0
        count_saved = 0
        success = True

        fps_savecheck_progress = 0

        # only extract a subset of frames
        while success and count_saved<=100:
        # extract all frames
            # save frame at desired framerate, indexing filename from 0
            if count_frame > fps_savecheck_progress:
                cv2.imwrite(path_data + filename + '/' + filename + "_%06d.jpg" % count_saved, image)
                fps_savecheck_progress += fps_savecheck
                count_saved += 1
            success,image = vidcap.read()
            count_frame += 1

        # close video file
        vidcap.release()
        
    else:
        print("Frames already extracted from video {}/{}".format(c+1,len(paths)))

# Done
print("Done extracting frames from {} videos".format(len(paths)))


## Create CSV file 

In [49]:
import re


In [97]:
def create_split(paths):
    file = [f for f in listdir(paths[0])  if isfile(join(paths[0], f))]
    file2 = [f for f in listdir(paths[1])  if isfile(join(paths[1], f))]
    merged=file+file2
    df = pd.DataFrame(columns=['video_name', 'tag'])
    print(df)
    for f in merged:
        new_row = {'video_name':str(f), 'tag':re.sub('[^a-zA-Z]+', '', str(f))}
        df = df.append(new_row, ignore_index=True)
        
    return df

In [98]:
df=create_split(['',''])

Empty DataFrame
Columns: [video_name, tag]
Index: []


In [99]:
df.sample(10)

,video_name,tag
186,no_wr_way_46.mp4,nowrwaymp
40,wr_way_22.mp4,wrwaymp
182,no_wr_way_95.mp4,nowrwaymp
76,wr_way_42.mp4,wrwaymp
57,wr_way_19.mp4,wrwaymp
49,wr_way_34.mp4,wrwaymp
161,no_wr_way_30.mp4,nowrwaymp
39,wr_way_6.mp4,wrwaymp
202,no_wr_way_44.mp4,nowrwaymp
17,wr_way_67.mp4,wrwaymp


In [100]:
df.shape

(203, 2)

In [101]:
df.to_csv('labeled.csv')

In [103]:
X=df.video_name
y=df.tag

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [105]:
train,test=pd.DataFrame(columns=['video_name', 'tag']),pd.DataFrame(columns=['video_name', 'tag'])
train['video_name'],train['tag']=X_train,y_train
test['video_name'],test['tag']=X_test,y_test

In [106]:
train

,video_name,tag
198,no_wr_way_93.mp4,nowrwaymp
93,wr_way_50.mp4,wrwaymp
38,wr_way_13.mp4,wrwaymp
24,wr_way_58.mp4,wrwaymp
96,wr_way_100.mp4,wrwaymp
...,...,...
106,no_wr_way_60.mp4,nowrwaymp
14,wr_way_99.mp4,wrwaymp
92,wr_way_44.mp4,wrwaymp
179,no_wr_way_94.mp4,nowrwaymp


In [107]:
test

,video_name,tag
15,.DS_Store,DSStore
9,wr_way_89.mp4,wrwaymp
115,no_wr_way_67.mp4,nowrwaymp
78,wr_way_81.mp4,wrwaymp
66,wr_way_55.mp4,wrwaymp
45,wr_way_5.mp4,wrwaymp
143,no_wr_way_5.mp4,nowrwaymp
177,no_wr_way_57.mp4,nowrwaymp
200,no_wr_way_78.mp4,nowrwaymp
180,no_wr_way_80.mp4,nowrwaymp


In [108]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [ ]:
pickle.dump(favorite_color, open("save.p", "wb"))